# Instalar dependencias

In [220]:
%pip install openai dotenv gradio PyMuPDF docx2txt

Note: you may need to restart the kernel to use updated packages.


# Importar librerias

In [221]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import gradio as gr
import fitz # PyMuPDF
import docx2txt

# Definir las variables de las API KEYS

In [222]:
load_dotenv(override=True)
open_router_api_key = os.getenv('OPEN_ROUTER_API_KEY')
gemini_api_key = os.getenv('GEMINI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

# Conectar los distintos modelos

## Google

In [223]:
gemini_model = dict(model=OpenAI(api_key=gemini_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/"), model_name="gemini-2.5-flash")

In [224]:
#gemini = OpenAI(api_key=gemini_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
#model_name = "gemini-2.5-flash"



# response = gemini_model["model"].chat.completions.create(model=gemini_model["model_name"], messages=[ # type: ignore
#     {
#       "role": "user",
#       "content": "Que es AI"
#     }
#   ])
# answer = response.choices[0].message.content
# answer

## OpenRouterAI

In [225]:
open_router_model = dict(model=OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=open_router_api_key,
), 
model_name="deepseek/deepseek-r1-0528")

In [226]:
#open_router_ai = OpenAI(
#  base_url="https://openrouter.ai/api/v1",
#  api_key=open_router_api_key,
#)

# open_router_model = dict(model=OpenAI(
#   base_url="https://openrouter.ai/api/v1",
#   api_key=open_router_api_key,
# ), 
# model_name="deepseek/deepseek-r1-0528")

# completion = open_router_model["model"].chat.completions.create(
#   #extra_headers={
#   #  "HTTP-Referer": "MI PAGINA o APP", 
#   #  "X-Title": "ANDY CODE",
#   #},
#   model=open_router_model["model_name"], # type: ignore
#   messages=[
#     {
#       "role": "user",
#       "content": "Que es AI"
#     }
#   ]
# )
# print(completion.choices[0].message.content)

## Ollama

In [227]:
# ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
# model_name = "llama3.2"

# response = ollama.chat.completions.create(model=model_name, messages=[
#     {
#       "role": "user",
#       "content": "Que es AI"
#     }
#   ])
# answer = response.choices[0].message.content
# answer

In [228]:
# from ollama import chat

# model_name = "llama3.2"
# mensajes = [
#     {"role": "user", "content": "¿Qué es la IA?"}
# ]

# respuesta = chat(model=model_name, messages=mensajes)
# print("Respuesta:", respuesta.message.content)


## Groq

In [229]:
groq_model = dict(model=OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1"), model_name="llama-3.3-70b-versatile")

In [230]:
#grog_ai = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
#model_name = "llama-3.3-70b-versatile"

# groq_model = dict(model=OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1"), model_name="llama-3.3-70b-versatile")

# response = groq_model["model"].chat.completions.create(model=groq_model["model_name"], messages=[
#     {
#       "role": "user",
#       "content": "Que es AI"
#     }
#   ])
# answer = response.choices[0].message.content
# answer

# Crear interfaz de chat con gradio

In [231]:
def test():
    if "C:\\pepe\\manduco" == r"C:\pepe\manduco":
        print("SON IGUALES")
    else:
        print("NO SON IGUALES")

test()

SON IGUALES


## Funcion de chat

In [234]:
def obtener_valor(dict: dict, key: str):
    return dict.get(key, "Clave no encontrada")

models = dict(
    gemini=gemini_model,
    open_router=open_router_model,
    groq=groq_model
)

sent_files = []

def chat(message: dict, history, model):

    try:

        print(f"Modelo seleccionado: {model}")
        print(f"modelo: {obtener_valor(models[model], 'model_name')}")

        messages = [
            {"role": "system", "content": "Eres un asistente de IA que responde preguntas y ayuda con tareas."}
        ]

        if message["files"]:
            sent_files.append(message["files"][0])
            for file in message["files"]:
                if os.path.basename(file).endswith('.pdf'):
                    file_content = read_file_pdf(file)
                elif os.path.basename(file).endswith('.txt'):
                    file_content = read_file_txt(file)
                elif os.path.basename(file).endswith('.docx'):
                    file_content = read_file_docx(file)
                else:
                    file_content = "Tipo de archivo no soportado. Solo se permiten archivos .txt, .pdf y .docx"
                
                # Añadir el contenido del archivo al historial como un mensaje del sistema
                history.append({"role": "system", "content": f"Contenido del archivo \"{os.path.basename(file)}\": {file_content}"})

        for msg in history:
            if valid_history(msg, sent_files[0] if sent_files else ""):
                messages.append({"role": msg["role"], "content": msg["content"]})
        
        messages.append({"role": "user", "content": message["text"]})

        resp = obtener_valor(models[model], "model").chat.completions.create(
            model=obtener_valor(models[model], "model_name"),
            messages=messages
        )

        return resp.choices[0].message.content
    except Exception as e:
        print(f"Error: {e}")
        return f"Error: {e}"
    
def valid_history(history, file):
    if len(history["content"]) > 0 and isinstance(history["content"], tuple):
        if history["content"][0] == file:
            return False
    return True
    
def read_file_txt(file):
    try:
        if file is None:
            return ""
        with open(file, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        return f"Error al leer el archivo: {e}"
    
def read_file_pdf(file):
    try:
        with fitz.open(file) as doc:
            text = ""
            for page in doc:
                text += page.get_text()
        return text
    except Exception as e:
        return f"Error al leer el archivo PDF: {e}"
    
def read_file_docx(file):
    try:
        texto = docx2txt.process(file)
        return texto
    except Exception as e:
        return f"Error al leer DOCX {file}: {e}"


## Creacion de la interfaz del Chat con Gradio

In [235]:
with gr.Blocks() as app:
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## Elija su modelo de IA preferido.")
            model_selector = gr.Dropdown(
                choices=list(models.keys()),
                value="gemini",
                label="Modelo de IA"
            )

        with gr.Column(scale=3):

            current_model = gr.State("gemini")

            model_selector.change(
                fn=lambda model: model,
                inputs=model_selector,
                outputs=current_model
            )

            chatbot = gr.Chatbot(type="messages")

            gr.ChatInterface(chat, 
                             chatbot=chatbot, 
                             additional_inputs=[current_model],
                             type="messages", 
                             title="Chat con Modelos de IA", 
                             multimodal=True,
                             save_history=True,
                             autoscroll=True,
                             stop_btn=True
                            )


app.launch()

* Running on local URL:  http://127.0.0.1:7958
* To create a public link, set `share=True` in `launch()`.
